# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298483634392                   -0.85    5.1
  2   -8.300209218973       -2.76       -1.25    1.2
  3   -8.300449419123       -3.62       -1.89    3.6
  4   -8.300462344425       -4.89       -2.76    1.2
  5   -8.300464216045       -5.73       -3.04    3.2
  6   -8.300464407559       -6.72       -3.21    1.0
  7   -8.300464521127       -6.94       -3.35    9.1
  8   -8.300464579093       -7.24       -3.47    2.5
  9   -8.300464627879       -7.31       -3.68    1.0
 10   -8.300464637674       -8.01       -3.84    1.0
 11   -8.300464643395       -8.24       -4.14    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67380399417                   -0.70    6.4
  2   -16.67791491638       -2.39       -1.14    1.0
  3   -16.67923453225       -2.88       -1.87    4.6
  4   -16.67928015554       -4.34       -2.71    3.5
  5   -16.67928564824       -5.26       -3.07    5.6
  6   -16.67928621174       -6.25       -3.49    1.2
  7   -16.67928622127       -8.02       -4.03    1.4


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32554073642                   -0.56    6.6
  2   -33.33266095451       -2.15       -1.00    1.1
  3   -33.33415313041       -2.83       -1.73    4.5
  4   -33.33528709182       -2.95       -2.29    3.0
  5   -33.33690602712       -2.79       -2.46   13.9
  6   -33.33693138673       -4.60       -2.72    2.0
  7   -33.33694380349       -4.91       -3.56    1.9
  8   -33.33694392037       -6.93       -3.89    4.5
  9   -33.33694392583       -8.26       -4.45    1.5


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298606582018                   -0.85    5.2
  2   -8.300287911855       -2.77       -1.58    1.0
  3   -8.300404957645       -3.93       -2.33    5.4
  4   -8.300311500025   +   -4.03       -2.16    3.8
  5   -8.300464383451       -3.82       -3.59    2.1
  6   -8.300464596934       -6.67       -3.96    3.2
  7   -8.300464640887       -7.36       -4.55    2.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32523090162                   -0.56    6.1
  2   -33.32234369092   +   -2.54       -1.27    1.0
  3   -20.62794641005   +    1.10       -0.52    8.0
  4   -33.29231914543        1.10       -1.55    8.1
  5   -33.26536650263   +   -1.57       -1.37    3.5
  6   -33.28405525430       -1.73       -1.70    3.5
  7   -33.11634646109   +   -0.78       -1.39    4.9
  8   -33.33660659354       -0.66       -2.49    4.4
  9   -33.33656897954   +   -4.42       -2.67    3.6
 10   -33.33638836254   +   -3.74       -2.46    2.6
 11   -33.33691080826       -3.28       -3.19    2.6
 12   -33.33694049817       -4.53       -3.55    3.4
 13   -33.33694292759       -5.61       -3.79    2.8
 14   -33.33694357272       -6.19       -4.05    2.2


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.